# Sanity Checks and Validation Tests

**Author:** Divyansh Atri

## Overview

This notebook performs comprehensive sanity checks on the implementation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from utils import *

print('Sanity Checks - Ready')

## Test 1: SDE Models

In [ ]:
# Test all models
models = [
    ('LQR', LinearQuadraticModel()),
    ('CBM', ControlledBrownianMotion()),
    ('MR', MeanRevertingModel())
]

x = np.array([0.5])
u = np.array([0.1])

print('Testing SDE models:')
for name, model in models:
    b = model.drift(x, u)
    sigma = model.diffusion(x)
    print(f'  {name}: b={b[0]:.4f}, σ={sigma[0]:.4f}')
    assert not np.isnan(b[0]), f'{name} drift is NaN'
    assert not np.isnan(sigma[0]), f'{name} diffusion is NaN'

print('✓ All models passed')

## Test 2: Cost Functions

In [ ]:
cost = QuadraticCost(q=1.0, r=1.0, q_terminal=10.0)

L = cost.running_cost(x, u)
g = cost.terminal_cost(x)

print(f'Running cost: L={L[0]:.4f}')
print(f'Terminal cost: g={g[0]:.4f}')

assert L[0] > 0, 'Running cost should be positive'
assert g[0] > 0, 'Terminal cost should be positive'

print('✓ Cost functions passed')

## Test 3: HJB Solver

In [ ]:
model = ControlledBrownianMotion(sigma=0.5)
cost_fn = QuadraticCost(q=1.0, r=1.0, q_terminal=10.0)

solver = HJBSolver(-2, 2, 51, 1.0, 51, model, cost_fn)
V, u_opt = solver.solve_backward(u_bounds=(-5, 5), verbose=False)

print(f'Value function shape: {V.shape}')
print(f'V(0, 0) = {V[0, 25]:.6f}')

assert V.shape == (51, 51), 'Wrong shape'
assert not np.any(np.isnan(V)), 'NaN in solution'
assert V[0, 25] > 0, 'Value should be positive'

print('✓ HJB solver passed')

## Test 4: HJB Residual

In [ ]:
residual_stats = verify_hjb_solution(V, solver.t, solver.x, model, cost_fn, u_opt, solver.dx, solver.dt)

print(f'HJB Residual Statistics:')
print(f'  Max abs residual: {residual_stats["max_abs_residual"]:.6e}')
print(f'  Mean abs residual: {residual_stats["mean_abs_residual"]:.6e}')
print(f'  RMS residual: {residual_stats["rms_residual"]:.6e}')

assert residual_stats['max_abs_residual'] < 1.0, 'Residual too large'

print('✓ Residual check passed')

## Test 5: Simulation

In [ ]:
policy_fn = lambda t, x: solver.get_policy(t, x)
sim = ClosedLoopSimulator(model, cost_fn, policy_fn)

results = sim.simulate(0.0, 1.0, 0.01, n_paths=100, seed=42)

print(f'Simulation results:')
print(f'  Mean cost: {results["mean_cost"]:.6f}')
print(f'  Std cost: {results["std_cost"]:.6f}')
print(f'  V(0,0): {V[0, 25]:.6f}')
print(f'  Difference: {abs(results["mean_cost"] - V[0, 25]):.6f}')

assert abs(results['mean_cost'] - V[0, 25]) < 1.0, 'Simulation mismatch'

print('✓ Simulation passed')

## Summary

All sanity checks passed! The implementation is working correctly.